# Data Preparing

In [1]:
import os
import csv
import random
import numpy as np
import pandas as pd

In [2]:
corpus_list = ['cejc','mpdd']
situation_list = ['apology','request','thanksgiving']
sen_type_list = ['query','res']
src_type = 'original' #'translated'
ver_name = '300_culturizefromT5train_all_both_prefix_rel'
context_len = 0

data_ver_name = '000_translate_all_both_prefix_rel'

data_dir = f'outputs/context/{data_ver_name}/{context_len}/'
bestmodel_dir = f'outputs/context/{ver_name}/{context_len}/best_model/'
save_dir = f'outputs/context/{ver_name}/{context_len}/'


In [3]:
def get_data_as_list(path):
    data = []
    with open(path, 'r', encoding='utf-8-sig')as f:
        reader = csv.reader(f)
        for row in reader:
            data.append(row[0])
    return data

def get_df(corpus_list, situation_list, sen_type_list, src_type, context_len, train_type):
    target_text = []
    input_text = []
    prefix = []
    for corpus in corpus_list:
        for situation in situation_list:
            for sen_type in sen_type_list:
                f_path = f'/nfs/nas-7.1/yamashita/LAB/dialogue_data/data/{corpus}/{situation}/{context_len}/rewrited_{sen_type}_{train_type}'
                target_text += get_data_as_list(f_path)
                
                f_path = f'/nfs/nas-7.1/yamashita/LAB/dialogue_data/data/{corpus}/{situation}/{context_len}/{src_type}_{sen_type}_{train_type}'
                input_text += get_data_as_list(f_path)
                
                f_path = f'/nfs/nas-7.1/yamashita/LAB/dialogue_data/data/{corpus}/{situation}/{context_len}/relation_pair_{train_type}'
                rel_list = []
                with open(f_path, 'r', encoding='utf-8-sig')as f:
                    reader = csv.reader(f)
                    for row in reader:
                        if sen_type == 'query':
                            rel_list.append(row[0])
                        elif sen_type == 'res':
                            rel_list.append(row[1])
                tmp_prefix = []
                for rel in rel_list:
                    tmp_prefix.append(f'{corpus} {situation} {sen_type} {rel}')     
                prefix += tmp_prefix
    df = pd.DataFrame([prefix,input_text,target_text], index=['prefix','input_text','target_text']).astype(str).T
    return df

# Make Dataset

In [4]:

import logging
import sacrebleu
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


model_args = T5Args()
model_args.max_length = 128
model_args.length_penalty = 20
model_args.num_beams = 10

model_ver_name = '000_translate_all_both_prefix_rel'
bestmodel_dir = f'outputs/context/{model_ver_name}/{context_len}/best_model/'

# model = T5Model("mt5", bestmodel_dir, args=model_args)
model = T5Model("mt5", bestmodel_dir, args=model_args, cuda_device=1)

In [5]:
train_type = 'train'  
train_df = get_df(corpus_list, situation_list, sen_type_list, src_type, context_len, train_type)

to_ja_truth = train_df.loc[train_df["prefix"].str.contains("mpdd")]["target_text"].tolist()
to_ja_input = train_df.loc[train_df["prefix"].str.contains("mpdd")]["input_text"].tolist()
to_ja_prefix = train_df.loc[train_df["prefix"].str.contains("mpdd")]["prefix"].tolist()

to_zh_truth = train_df.loc[train_df["prefix"].str.contains("cejc")]["target_text"].tolist()
to_zh_input = train_df.loc[train_df["prefix"].str.contains("cejc")]["input_text"].tolist()
to_zh_prefix = train_df.loc[train_df["prefix"].str.contains("cejc")]["prefix"].tolist()

to_ja_preds = model.predict(to_ja_input)
to_zh_preds = model.predict(to_zh_input)

input_text = to_ja_preds+to_zh_preds
train_df["input_text"] = input_text
target_text = to_ja_truth+to_zh_truth
train_df["target_text"] = target_text
prefix = to_ja_prefix+to_zh_prefix
train_df["prefix"] = prefix



/home/yamashita/anaconda3/envs/st/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3216: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  warnings.warn(


In [6]:
train_type = 'val'  
eval_df = get_df(corpus_list, situation_list, sen_type_list, src_type, context_len, train_type)

to_ja_truth = eval_df.loc[eval_df["prefix"].str.contains("mpdd")]["target_text"].tolist()
to_ja_input = eval_df.loc[eval_df["prefix"].str.contains("mpdd")]["input_text"].tolist()
to_ja_prefix = eval_df.loc[eval_df["prefix"].str.contains("mpdd")]["prefix"].tolist()

to_zh_truth = eval_df.loc[eval_df["prefix"].str.contains("cejc")]["target_text"].tolist()
to_zh_input = eval_df.loc[eval_df["prefix"].str.contains("cejc")]["input_text"].tolist()
to_zh_prefix = eval_df.loc[eval_df["prefix"].str.contains("cejc")]["prefix"].tolist()

to_ja_preds = model.predict(to_ja_input)
to_zh_preds = model.predict(to_zh_input)

input_text = to_ja_preds+to_zh_preds
eval_df["input_text"] = input_text
target_text = to_ja_truth+to_zh_truth
eval_df["target_text"] = target_text
prefix = to_ja_prefix+to_zh_prefix
eval_df["prefix"] = prefix

In [7]:
# train_df["prefix"] = ""
# eval_df["prefix"] = ""

display(train_df.iloc[:5])
display(eval_df.iloc[:5])

,prefix,input_text,target_text
0,mpdd apology query consignee,すみません。私は飲めないんです。でも飲めるなら、師傅に飲ませてもらえませんか?,すみません。本当に飲めないんです。飲めるなら絶対に飲んでますよ。
1,mpdd apology query enemy,すみません、我錯了。,ごめんなさい。本当にお兄さんに申し訳ない。
2,mpdd apology query couple,すみません、他のことは全部承知しています。ただ、この問題は私のせいです。,ごめん、他のことはともかく、これだけはだめなんだ。
3,mpdd apology query couple,ごめんなさい、兄を放してあげてくれない?,ごめん。兄を放してあげてくれない？
4,mpdd apology query friend,ごめんね。助けてもらえません。,やめてよ。悪いのは助けられない私だから。


,prefix,input_text,target_text
0,mpdd apology query couple,"抱歉,我不是故意的。",ごめんね、そんなつもりじゃなかったんだ。
1,mpdd apology query couple,"今天剛剛出來了,突然進去。",今日出てきたばかりなのに、また戻るなんて、本当に申し訳ないです。
2,mpdd apology query friend,すみません、我只帶了四萬元。,すみません、四万元持ってきましたん。
3,mpdd apology query friend,ごめんね。でも、私の関係は関係ありません。でも......。,謝るのは私の方だよ。そういう風に思わてるって知ってたんだから。でも、そういうのは二人のことだ...
4,mpdd apology query colleague,ごめんなさい、助けてもらえないんです。,それは力になれない。ごめんね。


# Finetune

In [8]:
model_args = T5Args()

model_args.max_seq_length = 128
model_args.length_penalty = 20
model_args.train_batch_size = 2
model_args.eval_batch_size = 2
model_args.num_train_epochs = 20
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 500
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.early_stopping_metric = 'eval_loss'
model_args.early_stopping_metric_minimize = True
model_args.early_stopping_patience = 3
model_args.use_early_stopping = True
model_args.save_eval_checkpoints = True
model_args.save_eval_checkpoints = False
model_args.learning_rate = 3e-5
model_args.best_model_dir = save_dir+'best_model/'
model_args.output_dir = save_dir+'ckpt/'
model_args.save_model_every_epoch = True
model_args.save_steps = -1
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = ver_name

# model = T5Model("mt5", "google/mt5-base", args=model_args)
model = T5Model("mt5", "google/mt5-base", args=model_args, cuda_device=1)
# Train the model
os.environ['WANDB_CONSOLE'] = 'off'
model.train_model(train_df, eval_data=eval_df)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


/home/yamashita/anaconda3/envs/st/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3216: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  warnings.warn(
INFO:simpletransformers.t5.t5_model: Training started



Using Adafactor for T5


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: natsukinateyamashita (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.33 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/yamashita/anaconda3/envs/st/lib/python3.9/site-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/home/yamashita/anaconda3/envs/st/lib/python3.9/site-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/home/yamashita/anaconda3/envs/st/lib/python3.9/site-packages/transformers/o

...

INFO:simpletransformers.t5.t5_model: No improvement in eval_loss
INFO:simpletransformers.t5.t5_model: Current step: 3
INFO:simpletransformers.t5.t5_model: Early stopping patience: 3
INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


/home/yamashita/anaconda3/envs/st/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3216: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  warnings.warn(


INFO:simpletransformers.t5.t5_model: Patience of 3 steps reached
INFO:simpletransformers.t5.t5_model: Training terminated.


INFO:simpletransformers.t5.t5_model: Training of google/mt5-base model complete. Saved to outputs/context/300_culturizefromT5train_all_both_prefix_rel/0/ckpt/.


(23000,
 {'global_step': [500,
   1000,
   1318,
   1500,
   2000,
   2500,
   2636,
   3000,
   3500,
   3954,
   4000,
   4500,
   5000,
   5272,
   5500,
   6000,
   6500,
   6590,
   7000,
   7500,
   7908,
   8000,
   8500,
   9000,
   9226,
   9500,
   10000,
   10500,
   10544,
   11000,
   11500,
   11862,
   12000,
   12500,
   13000,
   13180,
   13500,
   14000,
   14498,
   14500,
   15000,
   15500,
   15816,
   16000,
   16500,
   17000,
   17134,
   17500,
   18000,
   18452,
   18500,
   19000,
   19500,
   19770,
   20000,
   20500,
   21000,
   21088,
   21500,
   22000,
   22406,
   22500,
   23000],
  'eval_loss': [9.665996354998963,
   6.188658883354881,
   5.178102623332631,
   4.847697860905618,
   4.21884436787981,
   3.861632505691413,
   3.8062425223263827,
   3.690254407940489,
   3.542781295740243,
   3.433526810010274,
   3.421010955129609,
   3.386646724379424,
   3.306777390747359,
   3.299315007437359,
   3.277421881213333,
   3.272622605164846,
   3.216

# Test

In [9]:

import logging
import sacrebleu
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


model_args = T5Args()
model_args.max_length = 128
model_args.length_penalty = 20
model_args.num_beams = 10

# model = T5Model("mt5", save_dir+"best_model/", args=model_args)
model = T5Model("mt5", save_dir+"best_model/", args=model_args, cuda_device=1)

In [10]:

ja_eval_df = pd.read_csv(f"{data_dir}ja_preds_truth.csv").astype(str)
zh_eval_df = pd.read_csv(f"{data_dir}zh_preds_truth.csv").astype(str)

to_ja_truth = ja_eval_df["truth"].tolist()
to_ja_input = ja_eval_df[f"{data_ver_name}"].tolist()

to_zh_truth = zh_eval_df["truth"].tolist()
to_zh_input = zh_eval_df[f"{data_ver_name}"].tolist()

train_type = 'test'  
eval_df = get_df(corpus_list, situation_list, sen_type_list, src_type, context_len, train_type)

to_ja_prefix = eval_df.loc[eval_df["prefix"].str.contains("mpdd")]["prefix"].tolist()
to_zh_prefix = eval_df.loc[eval_df["prefix"].str.contains("cejc")]["prefix"].tolist()

# to_ja_input = [": " + input_text for input_text in to_ja_input]
# to_zh_input = [": " + input_text for input_text in to_zh_input]
to_ja_input = [prefix + ": " + input_text for prefix, input_text in zip(to_ja_prefix, to_ja_input)]
to_zh_input = [prefix + ": " + input_text for prefix, input_text in zip(to_zh_prefix, to_zh_input)]
to_zh_input[:5]



['cejc apology query consignor: 不會啦,抱歉,兩合分了。',
 'cejc apology query consignee: 不好意思麻煩你了。',
 'cejc apology query friend: 真的假的?啊,不好意思。',
 'cejc apology query friend: 嗯?抱歉,我剛剛沒有注意到。',
 'cejc apology query consignor: 不好意思,請您稍等一下。']

In [11]:
to_ja_preds = model.predict(to_ja_input)
to_zh_preds = model.predict(to_zh_input)

/home/yamashita/anaconda3/envs/st/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3216: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  warnings.warn(


In [12]:
r_ja_df = pd.DataFrame([to_ja_preds,to_ja_truth],index=[f'{ver_name}', 'truth'])
r_ja_df.T.to_csv(save_dir+'ja_preds_truth.csv',encoding='utf_8_sig')

r_zh_df = pd.DataFrame([to_zh_preds,to_zh_truth],index=[f'{ver_name}', 'truth'])
r_zh_df.T.to_csv(save_dir+'zh_preds_truth.csv',encoding='utf_8_sig')